In [16]:
from bs4 import BeautifulSoup
import requests as rq
import re
import datetime
import pymysql
from flask import Flask
from flask import jsonify, request
from fake_useragent import UserAgent
import cv2
import numpy as np
import time
from urllib.parse import urlencode
from flask import Flask,request
import threading
import sys
import os
import json
import shutil
import traceback
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [17]:
#lxml pyinstaller編不動，nuitka不行
#parser = "lxml"
parser = "html.parser"
parser_detail = 'html5lib'

In [73]:
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "As123459362",
    "db": "pttcrawler",
    "charset": "utf8",
    "autocommit":True
}
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "29th.cc",  #目標網站 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
     #使用者代理
}

In [19]:
# 從免費代理網站取得代理ip
def get_proxy_ip(isHttps):
    url = {"usproxy":"https://www.us-proxy.org/","freeproxylist":"https://free-proxy-list.net/"}
    metas=[]
    for i in url:
        r = rq.get(url[i],verify=False)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, parser)
        trs = soup.select("table.table.table-striped.table-bordered tr")  
        for tr in trs:
            tds = tr.select("td")
            if len(tds) > 6:
                ifScheme = tds[6].text
                ip = tds[0].text
                port = tds[1].text
                anonymity = tds[4].text
                proxy = "%s:%s"%(ip, port)
                meta = {
                    'proxyIp': proxy,
                    'connect_times':0,
                    'successful_connect_times':0,
                    'qual_ratio':0,
                    'isDelete':False,
                    'fromm':i
                  }
                if ifScheme == isHttps:
                    if ifScheme == "yes":
                        meta['ishttps'] = True
                        # https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#https-proxy-error-http-proxy
                        meta['proxyIp'] = "http://" + meta['proxyIp']
                    elif ifScheme == "no":
                        meta['ishttps'] = False
                        meta['proxyIp'] = "http://" + meta['proxyIp']
                    meta = test_proxy(meta,timeout_sec=(4,7))
                    metas.append(meta)
                    continue
    return metas

In [20]:
# 測試 proxy 可用性
def test_proxy(proxy,timeout_sec,headers={}):
    url = "http://icanhazip.com/"
    print(proxy['proxyIp'])
    try:
        if proxy['ishttps'] == True:
            r = rq.get(url, headers = headers,proxies={'http':proxy['proxyIp'],},verify=False, timeout=timeout_sec)
        elif proxy['ishttps'] == False:
            r = rq.get(url, headers = headers,proxies={'http':proxy['proxyIp'],},verify=False, timeout=timeout_sec)
        print(r.status_code)
        print(r.text.strip())
        if r.status_code==200:
            proxy['isValidate']=True
    except Exception as ex:
        proxy['isValidate']=False
        print(ex)
    print("---------------")
    return proxy

In [21]:
# 將取得的代理ip寫入資料庫，此函數也可以更新已寫入資料庫的資料
def send_ip(proxies_pool_https, db_settings,typee="",send_log = True):
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    wrong = 0
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id="select MAX(UID) FROM log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = "INSERT INTO proxy_ip(log_UID, proxyIp, connect_times, successful_connect_times, qual_ratio, isValidate, ishttps,isDelete, fromm)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),connect_times =values(connect_times),successful_connect_times=values(successful_connect_times),qual_ratio=values(qual_ratio),isValidate = values(isValidate),ishttps=values(ishttps),isDelete=values(isDelete)"
            command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始
            if send_log:
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "start", typee, ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 組合數據
            if type(proxies_pool_https) == list:
                for proxy in proxies_pool_https:
                    data_combine.append((log_UID, proxy["proxyIp"], proxy["connect_times"], proxy["successful_connect_times"], proxy["qual_ratio"], proxy["isValidate"], proxy["ishttps"], proxy["isDelete"], proxy["fromm"]))  # 注意要用两个括号扩起来
            else:
                data_combine.append((log_UID, proxies_pool_https["proxyIp"], proxies_pool_https["connect_times"], proxies_pool_https["successful_connect_times"], proxies_pool_https["qual_ratio"], proxies_pool_https["isValidate"], proxies_pool_https["ishttps"], proxies_pool_https["isDelete"], proxies_pool_https["fromm"]))  # 注意要用两个括号扩起来
            # 執行
            try:
                cursor.executemany(command, data_combine)
            except Exception as err:
                print(err)
                wrong = wrong + 1
                print(wrong)
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "wrong" + str(wrong), "", str(err)))
            # 紀錄結束
            if wrong==0 and send_log:
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "finish", "well_completed", ""))
            elif wrong>0 and send_log:
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "finish", str(wrong) + " records are wrong", ""))
    except Exception as ex:
        print(ex)

    time_end = datetime.datetime.now()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('time cost', time_c, 's')

In [22]:
# 從資料庫取出ip，未來有自架proxy的話，可以把proxy放到資料庫內，並設定 mine = 1，這樣就會固定抓這個proxy來用，如果沒有自架proxy的話，就還需要多寫 proxy expire 的處置方式。
def get_ip_from_db(isHttps,db_settings,qual_ratio=0,connect_times=4):
    try:
        # 建立Connection物件
        data=[]
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            if isHttps=="yes":
                get_ip=f"select * FROM proxy_ip where isHttps = 1 and isDelete is False and ( qual_ratio > {qual_ratio} or ( connect_times < {connect_times} )) or (mine = 1 and isHttps = 1)"
            elif isHttps=="no":
                get_ip=f"select * FROM proxy_ip where isHttps = 0 and isDelete is False and ( qual_ratio > {qual_ratio} or ( connect_times < {connect_times} )) or (mine = 1 and isHttps = 0)"
            cursor.execute(get_ip)
            xx = cursor.fetchall()
            des = cursor.description
            
            for i in xx:
                d = {}
                x=0
                for j in des:
                    d[j[0]]=i[x]
                    x=x+1
                data.append(d)
    except Exception as ex:
        print(ex)
    return data

In [3]:
conn = pymysql.connect(**db_settings)
# 建立Cursor物件
with conn.cursor(pymysql.cursors.DictCursor) as cursor:
  #資料表相關操作

    get_max_log_id="SELECT * FROM pttcrawler.thzu_data where nos_of_d =0 "
    cursor.execute(get_max_log_id)
    raw_datas = cursor.fetchall()            
print(f'一共抓出 {len(raw_datas)} 筆還沒有內頁資料')

一共抓出 142034 筆還沒有內頁資料


In [24]:
raw_datas[0:1]

[{'UID': 1,
  'title': 'MEYD-721 左右鄰居人妻抗議我A片太大聲卻被我3P狂插幹上癮還... (中文字幕)',
  'date': datetime.datetime(2022, 2, 3, 0, 0),
  'url': 'http://29th.cc/thread-2452832-1-1.html',
  'board': '有码',
  'page': 1,
  'log_UID': '1',
  'nos_of_d': 0,
  'type': '日韩情色(BT)',
  'look': 116,
  'reply': 0}]

In [15]:
proxies_this = get_ip_from_db("no",db_settings,qual_ratio=0,connect_times=4)
proxies_this[0]

{'proxyIp': '103.214.109.70:80',
 'log_UID': 227,
 'connect_times': 0,
 'successful_connect_times': 0,
 'qual_ratio': Decimal('0'),
 'isValidate': 1,
 'isDelete': 0,
 'ishttps': 0,
 'fromm': None,
 'mine': None}

In [84]:
proxies_this={}
proxies_all = get_ip_from_db("no",db_settings,qual_ratio=0,connect_times=4)
try:
    for i in raw_datas:
        print("-------------")
        with rq.session() as s:
            try:
                r = s.get(i['url'],headers=headers)
                html_doc = r.text
                soup = BeautifulSoup(html_doc, parser)
                trs = soup.select("table tr em.xg1")
                if trs !=[]:
                    i['nos_of_d']=int(re.search(r"下载次数: (.*)\)",trs[0].text).group(1))
                    continue
                trs = soup.select("dd p")
                if trs !=[]:
                    i['nos_of_d']=int(re.search(r"下载次数: (.*)",trs[3].text).group(1))
                    continue
            except requests.exceptions.RequestException as ex:
                print(ex)
                proxies_this={'http':proxies_all[0]['proxyIp']}
                proxies_all.pop(0)
            except Exception as ex:
                print(ex)
                print(i['url'])
except

-------------


In [67]:
t1 = threading.Thread(target=main, args=(url_list1, 1))
t_list.append(t1)
t2 = threading.Thread(target=main, args=(url_list2, 2))
t_list.append(t2)
t3 = threading.Thread(target=main, args=(url_list3, 3))
t_list.append(t3)

# 開始工作
for t in t_list:
    t.start()

HTTPConnectionPool(host='10.30.23.112', port=8888): Max retries exceeded with url: http://29th.cc/thread-2452832-1-1.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208EF2AD8E0>, 'Connection to 10.30.23.112 timed out. (connect timeout=1)'))


In [83]:
r = rq.get("http://29th.cc/thread-2452804-1-1.html",headers=headers)
html_doc = r.text
soup = BeautifulSoup(html_doc, "lxml")
trs = soup.select("dd p")
int(re.search(r"下载次数: (.*)",trs[3].text).group(1))


3126

In [81]:
trs[3]

<p>106.77 KB, 下载次数: 3126</p>

In [63]:
re.search(r"下载次数: (.*)\)",trs[0].text).group(1)

'2349'